In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os, sys
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *

In [12]:
model_df=pd.read_excel("../../data/250224 model_STD.xlsx")
maker_df=pd.read_excel("../../data/250224 maker_STD.xlsx")


In [ ]:
maker_df

,makerID,makerSTDName,makerAlias,CLEAN_MAKER_NM
0,MA273101,Bomyeong Hitech,Bomyeong Hitech,BOMYEONGHITECH
1,MA1538,Nexstar,"NEX Star,Nexstar",NEXSTAR
2,MA298455,NAMYOUNG,NAMYOUNG,NAMYOUNG
3,MA298457,"DHE CO.,LTD","DHE CO.,LTD",DHECO.LTD
4,MA278365,SAFEX,SAFEX,SAFEX
...,...,...,...,...
3778,MA999,ISOTECH,ISOTECH,ISOTECH
3779,MA973,INTERGEN,INTERGEN,INTERGEN
3780,MA932,Iglika,Iglika,IGLIKA
3781,MA97,ALPHA AUTOMATION INC,"ALPHA,ALPHA AUTOMATION INC",ALPHAAUTOMATIONINC


In [17]:
# 특수 예외 패턴 보정: 쉼표 제거 또는 대체
maker_df["makerAlias"] = maker_df["makerAlias"].str.lower()
maker_df["makerAlias"] = maker_df["makerAlias"].str.replace("co.,ltd", "co.ltd", regex=False)
maker_df["makerAlias"] = maker_df["makerAlias"].str.replace("co., ltd", "co.ltd", regex=False)
explode_model_df=alias_info_dataframe(model_df)
explode_maker_df=alias_info_dataframe(maker_df,"makerAlias")

In [32]:
explode_maker_df["makerAlias"]=explode_maker_df["makerAlias"].apply(clean_unknown_to_empty)

In [34]:
explode_maker_df.drop_duplicates(subset=["makerAlias","makerSTDName"], inplace=True)

In [39]:
explode_maker_df.to_csv("../../data/test123.csv", index=False)
explode_maker_df=pd.read_csv("../../data/test123.csv")
explode_maker_df.dropna(inplace=True)

In [40]:


# 1. 데이터 준비
data = explode_maker_df
label_encoder = LabelEncoder()
data['label_id'] = label_encoder.fit_transform(data['makerSTDName'])


# 2. 커스텀 Dataset 정의
class AliasDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 3. 학습 데이터 분리
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['makerAlias'].tolist(), data['label_id'].tolist(), test_size=0.1, random_state=42)

train_dataset = AliasDataset(train_texts, train_labels, tokenizer)
val_dataset = AliasDataset(val_texts, val_labels, tokenizer)

# 4. 모델 불러오기
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_encoder.classes_)
)

# 5. 학습 설정
training_args = TrainingArguments(
    output_dir="./bert_alias_model",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 6. 학습 시작
trainer.train()

# 7. 저장
tokenizer.save_pretrained("../model/bert_alias_model")
model.save_pretrained("../model/bert_alias_model")

# 8. 예측 함수
def predict_alias(text):
    encoding = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    with torch.no_grad():
        output = model(**encoding)
    pred = torch.argmax(output.logits, dim=1).item()
    return label_encoder.inverse_transform([pred])[0]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/workspace/Large_languige_model/.venv/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,7.716000,7.567528
2,7.288000,7.573235
3,7.152700,7.474590
4,6.959700,7.607539
5,6.679000,7.524020
6,6.591300,7.533133
7,6.403400,7.583707
8,6.203800,7.508178
9,6.218700,7.431211
10,6.040800,7.429640


In [25]:
# train 데이터 셋에서 첫번째데이터 보여줘
train_texts[0], train_labels[0]

('Gasonics', 1140)

In [22]:
def predict_alias(text):
    encoding = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    
    # 모델 디바이스 확인
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 입력 텐서도 같은 디바이스로 이동
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)

    pred = torch.argmax(output.logits, dim=1).item()
    return label_encoder.inverse_transform([pred])[0]
